This code amalgamates the summary information from the 'mylocalschool.wales' website into one large dictionary for easy access. 

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np

This is a list of the schools in the local area that we want to focus on.

In [2]:
local_schools = ['Severn Primary','St. Mary\'s R.C. Primary School','St.Paul\'s C/W Primary School','Grangetown Primary School','St Patrick\'s R C School','Ysgol Gynradd Gymraeg Hamadryad','Ninian Park Primary School','Kitchener Primary School','Radnor Primary School','Lansdowne Primary School','Ysgol Treganna','Ysgol Gymraeg Pwll Coch']

This function makes a dictionary which associates each school name to the URL of its summary page.

In [3]:
def make_school_URL_dict(schools):
    toppage = requests.get("http://mylocalschool.wales.gov.uk/Schools/SchoolSearch?lang=en")
    soup = BeautifulSoup(toppage.content, 'html.parser')
    school_links = soup.select("a[href*=/School]")[2:]
    schools = [list(school.children)[0] for school in school_links]
    school_URLs = ["http://mylocalschool.wales.gov.uk" + school['href'] for school in school_links]
    schoolURL_dict = {}
    for s,school in enumerate(schools):
        schoolURL_dict[school] = school_URLs[s]
    return schoolURL_dict

In [4]:
schoolURL_dict = make_school_URL_dict(local_schools)

This next function takes the URL of a schools summary page and makes a dictionary listing the stats for that school.

In [5]:
def tofloat(string):
    try:
        output = float(string)
    except ValueError:
        output = string
    return output
                
def make_stat_dict(schoolURL):
    school_pages = requests.get(schoolURL)
    school_soup = BeautifulSoup(school_pages.content, 'html.parser')
    summary = school_soup.find_all('div',id=False, class_="summaryBox")
    stat_dict = {}
    for i in range(len(summary)):
        stat = ''.join(char for char in list(summary[i].children)[1].getText() if char.isalnum() or char=='.')
        stat = tofloat(stat)
        stat_name = (''.join(char for char in list(summary[i].children)[3].getText() if char.isalnum() or char==' ')).strip()
        if '%' in list(summary[i].children)[1].getText(): stat_name += "(%)"
        stat_dict[stat_name] = stat
    return stat_dict

This dictionary links each school to the summary dictionary for that school.

In [6]:
local_school_dict = {}
for school in local_schools:
    local_school_dict[school] = make_stat_dict(schoolURL_dict[school])

Now we use pandas to turn this dictionary into a data frame.

In [7]:
import pandas as pd

In [8]:
local_school_df = pd.DataFrame.from_dict(local_school_dict, orient='index')

In [9]:
local_school_df

,Number of Pupils 2017,Free school meals FSM 3 year average Primary only(%),Pupil Teacher Ratio PTR Primary only,Attendance during the year Primary only(%),School budget per pupil,Pupils who have reached the expected level Core subject indicator Key Stage 2(%),Support Category,Pupils achieving the expected outcome in the Foundation Phase areas of learning(%),Average number of minutes per week allocated for curricular PE Primary only,of pupils in the school who enjoy PE lessons a lot(%),of pupils in the school who are hooked on sport(%)
Grangetown Primary School,404.0,27.2,19.6,94.2,3623.0,80.0,Yellow,76.3,NaN,NaN,NaN
Kitchener Primary School,486.0,29.9,20.5,94.5,3576.0,82.1,Green,80.8,105.0,74.0,26.0
Lansdowne Primary School,486.0,29.6,21.9,93.1,3442.0,85.1,Green,92.0,90.0,NaN,NaN
Ninian Park Primary School,557.0,23.7,24.3,94.1,3569.0,92.5,Green,84.0,NaN,61.0,32.0
Radnor Primary School,323.0,19.1,18.9,95.6,3862.0,100.0,Green,92.5,120.0,NaN,NaN
Severn Primary,482.0,24.0,20.9,93.9,3947.0,87.5,Green,85.5,90.0,NaN,NaN
St Patrick's R C School,298.0,23.2,24.4,94.0,3548.0,91.7,Yellow,95.3,NaN,NaN,NaN
St. Mary's R.C. Primary School,261.0,11.1,21.5,95.7,3388.0,92.3,Yellow,100.0,NaN,NaN,NaN
St.Paul's C/W Primary School,209.0,26.5,23.1,94.7,3774.0,92.9,Yellow,89.7,120.0,74.0,50.0
Ysgol Gymraeg Pwll Coch,506.0,9.5,22.6,95.8,3213.0,98.3,Yellow,95.2,NaN,NaN,NaN


Here we show a summary of the data.

In [10]:
local_school_df.describe()

,Number of Pupils 2017,Free school meals FSM 3 year average Primary only(%),Pupil Teacher Ratio PTR Primary only,Attendance during the year Primary only(%),School budget per pupil,Pupils who have reached the expected level Core subject indicator Key Stage 2(%),Pupils achieving the expected outcome in the Foundation Phase areas of learning(%),Average number of minutes per week allocated for curricular PE Primary only,of pupils in the school who enjoy PE lessons a lot(%),of pupils in the school who are hooked on sport(%)
count,12.000000,11.000000,12.000000,11.000000,12.000000,11.000000,11.000000,5.0,3.000000,3.000000
mean,387.666667,20.754545,21.600000,94.781818,4897.166667,90.354545,89.200000,105.0,69.666667,36.000000
std,171.661577,8.651285,2.355651,1.121444,4583.328680,6.206990,6.978682,15.0,7.505553,12.489996
min,17.000000,4.500000,17.000000,93.100000,3213.000000,80.000000,76.300000,90.0,61.000000,26.000000
25%,288.750000,15.100000,20.275000,94.050000,3428.500000,86.300000,84.750000,90.0,67.500000,29.000000
50%,443.000000,23.700000,21.700000,94.500000,3572.500000,91.700000,89.900000,105.0,74.000000,32.000000
75%,491.000000,26.850000,23.400000,95.650000,3796.000000,92.700000,93.850000,120.0,74.000000,41.000000
max,623.000000,29.900000,24.500000,97.000000,19436.000000,100.000000,100.000000,120.0,74.000000,50.000000


In [11]:
local_school_df.keys()

Index(['Number of Pupils 2017',
       'Free school meals FSM  3 year average Primary only(%)',
       'Pupil Teacher Ratio PTR Primary only',
       'Attendance during the year Primary only(%)', 'School budget per pupil',
       'Pupils who have reached the expected level  Core subject indicator Key Stage 2(%)',
       'Support Category',
       'Pupils achieving the expected outcome in the Foundation Phase areas of learning(%)',
       'Average number of minutes per week allocated for curricular PE Primary only',
       'of pupils in the school who enjoy PE lessons a lot(%)',
       'of pupils in the school who are hooked on sport(%)'],
      dtype='object')

The total number of primary school students in the area is:

In [15]:
local_num_students = sum(local_school_df['Number of Pupils 2017'])
print(total_num_students)

4652.0


Here we use the number of primary school students and the percentage on free school meals to estimate the total number of children in the area who are on free school meals.

In [16]:
local_fsm_students = sum((pd.to_numeric(local_school_df['Free school meals FSM  3 year average Primary only(%)'])/100 \
                                                  * local_school_df['Number of Pupils 2017']).dropna())
print(fsm_students)

938.0369999999999


The percentage of primary school students in the area on free school meals is:

In [17]:
local_fsm_students/local_num_students * 100

20.164165950128975

We will now compare the local school statistics to the whole of Cardiff.

In [ ]:
def make_school_URL_dict_for_district(district):
    toppage = requests.get("http://mylocalschool.wales.gov.uk/Schools/SchoolSearch?lang=en")
    soup = BeautifulSoup(toppage.content, 'html.parser')
    school_links = soup.select("a[href*=/School]")[2:]
    schools = [list(school.children)[0] for school in school_links]
    school_URLs = ["http://mylocalschool.wales.gov.uk" + school['href'] for school in school_links]
    schoolURL_dict = {}
    for s,school in enumerate(schools):
        schoolURL_dict[school] = school_URLs[s]
    return schoolURL_dict

In [188]:
def make_district_school_URLdict(district): 
    toppage = requests.get("http://mylocalschool.wales.gov.uk/Schools/SchoolSearch?lang=en")
    soup = BeautifulSoup(toppage.content, 'html.parser')
    url_ref = [list(soup.find_all("option"))[-1]['value'] for i in range(len(list(soup.select('option')))) \
                                            if list(soup.find_all("option")[i])[0].strip() == district][0]
    school_url_string = "a[href*=/School/" + url_ref + "]"
    school_links = soup.select(school_url_string)
    schools = [list(school.children)[0] for school in school_links]
    school_URLs = ["http://mylocalschool.wales.gov.uk" + school['href'] for school in school_links]
    schoolURL_dict = {}
    for s,school in enumerate(schools):
        schoolURL_dict[school] = school_URLs[s]
    return schoolURL_dict


In [189]:
district_school_URLdict = make_district_school_URLdict('Cardiff')

In [ ]:
cardiff_school_dict = {}
for school in cardiff_schools:
    cardiff_school_dict[school] = make_stat_dict(schoolURL_dict[school])